# OLS, Testing

This notebook estimates a linear regression and tests various hypotheses using standard errors assuming *(a)* iid residuals (Gauss-Markov assumptions); *(b)* heteroskedasticity (White); *(c)* autocorrelation and heteroskedasticity (Newey-West).

You may also consider the [HypothesisTests.jl](https://github.com/JuliaStats/HypothesisTests.jl) package (not used here).

## Load Packages and Extra Functions

In [1]:
using Printf, DelimitedFiles, Statistics, LinearAlgebra, Distributions

include("jlFiles/printmat.jl")
include("jlFiles/Ols.jl");         #functions for OLS

## Loading Data

In [2]:
x = readdlm("Data/FFmFactorsPs.csv",',',skipstart=1)

                #yearmonth, market, small minus big, high minus low
(ym,Rme,RSMB,RHML) = (x[:,1],x[:,2]/100,x[:,3]/100,x[:,4]/100)
x = nothing

printlnPs("Sample size:",size(Rme))

Sample size:    (388,)


## OLS under the Gauss-Markov Assumptions

(assuming iid residuals)

In [3]:
Y = Rme
T = size(Y,1)
X = [ones(T) RSMB RHML]

(b,u,_,V,R²) = OlsGMFn(Y,X)
std_iid = sqrt.(diag(V))

printblue("OLS Results (assuming iid residuals):\n")
xNames = ["c","SMB","HML"]
printmat(b,std_iid;colNames=["b","std_iid"],rowNames=xNames)

OLS Results (assuming iid residuals):

            b   std_iid
c       0.007     0.002
SMB     0.217     0.073
HML    -0.429     0.074



# Testing a Joint Hypothesis

Since the estimator $\hat{\beta}_{_{k\times1}}$ satisfies

$
\hat{\beta}-\beta_{0} \sim N(0,V_{k\times k})  ,
$

we can easily apply various tests. Consider a joint linear hypothesis of the
form

$
H_0: R\beta=q,
$

where $R$ is a $J \times k$ matrix and $q$ is a $J$-vector. To test this, use

$
(R\beta-q)^{\prime}(RVR^{\prime}) ^{-1}(R\beta
-q)\overset{d}{\rightarrow}\chi_{J}^{2}.
$

How we estimate $V$ depends on whether there is heteroskedasticity and/or autocorrelation (discussed below).

In [4]:
R = [0 1 0;               #testing if b₂=0 and b₃=0
     0 0 1]
q = [0;0]
test_stat = (R*b-q)'inv(R*V*R')*(R*b-q)    #R*V*R' is 2x2

printblue("Testing Rb = a:")
printmat([test_stat,quantile(Chisq(2),0.9)];rowNames=["test statistic","10% critical value"])

Testing Rb = a:
test statistic        60.010
10% critical value     4.605



# Distribution of OLS Estimates without the Gauss-Markov Assumptions


The distribution of the OLS estimates is (typically)

$
(\hat{\beta}-\beta_{0})\overset{d}{\rightarrow}N(0,V)
\: \text{ where } \: V=S_{xx}^{-1} S S_{xx}^{-1}.
$

and where $S_{xx} = \sum\nolimits_{t=1}^{T}x_{t}x_{t}^{\prime}$ 
and $S$ is the covariance matrix of $\sum_{t=1}^{T}u_{t}x_{t}$.


*When* the Gauss-Markov assumptions hold, then $S$ can be simplified to $S_{xx}\sigma^2$, where $\sigma^2$ is the variance of $u_t$, so $V=S_{xx}^{-1}\sigma^2$.

In contrast, with heteroskedasticity and/or autocorrelation, $S$ must be estimated differently.

# White's Covariance Matrix

If $u_{t}x_{t}$ is not autocorrelated, then $S$ simplifies to $\sum_{t=1}^{T} x_tx_t^{\prime}\sigma_t^2$. White's method replaces $\sigma_t^2$ by $\hat{u}_{t}^{2}$. This estimate is robust to heteroskedasticity (in particular, time variation in $\sigma_t^2$ that is related to $x_t$).

## A Remark on the Code
$S_{xx}$ can be calculated as `Sxx = X'X` and $S$ as `S = (X.*u)'*(X.*u)`.

Clearly, these calculations can also be done in a loop like
```
for t = 1:T
   Sxx = Sxx + X[t,:]*X[t,:]' 
   S   = S   + X[t,:]*X[t,:]'*u[t]^2
end
```

In [5]:
Sxx = X'X

S     = (X.*u)'*(X.*u)                #S according to White's method
V     = inv(Sxx)'S*inv(Sxx)           #Cov(b), White
std_W = sqrt.(diag(V))

printblue("Coefficients and standard errors (from different methods):\n")
xx = [b std_iid std_W]
printmat(xx;colNames=["b","std_iid","std_White"],rowNames=xNames,width=12)

Coefficients and standard errors (from different methods):

              b     std_iid   std_White
c         0.007       0.002       0.002
SMB       0.217       0.073       0.113
HML      -0.429       0.074       0.097



# Newey-West's Covariance Matrix

Let $g_t=u_{t}x_{t}$ be a $k$-vector of data.

To calculate the Newey-West covariance matrix, we first need

$ 
\Lambda_{s} = \sum_{t=s+1}^{T} (g_{t}-\bar{g})(g_{t-s}-\bar{g})^{\prime},
$

which is proportional to the $s$th autocovariance matrices.

Then we form a linear
combination (with tent-shaped weights) of those autocovariance matrices (from
lag $-m$ to $m$) as in

$
S = \mathrm{Cov}(\sum_t g_t)  = 
\Lambda_{0} + \sum_{s=1}^{m}( 1-\frac{s}{m+1})  
(\Lambda_{s}+\Lambda_{s}^{\prime}).
$

With $m=0$ this is the same as White's method.

If we divide $S$ by $T$, then we get an estimate of $\mathrm{Cov}(\sqrt{T} \bar{g})$, and if we instead divide by $T^2$ then we get an estimate of $\mathrm{Cov}(\bar{g})$.

The `CovNWFn()` function (below and also in `CovNWFn.jl`) implements this. 

## A Remark on the Code
$\Lambda_{s}$  can be calculated by `g[s+1:T,:]'g[1:T-s,:]` where `g` is already demeaned. (A loop would also work.)


In [6]:
"""
    CovNWFn(g0,m=0,DivideByT=0)

Calculates covariance matrix of sample sum (DivideByT=0), scaled average (DivideByT=1) or
average (DivideByT=2).


# Input
- `g0::Matrix`:      Txq Matrix of q moment conditions
- `m::Int`:          number of lags to use
- `DivideByT::Int`:  divide the result by T^DivideByT

# Output
- `S::Matrix`: qxq covariance matrix

# Remark
- `DivideByT=0`: Var(x_1+x_2+...X_T)
- `DivideByT=1`: Var((x_1+x_2+...X_T)/sqrt(T))
- `DivideByT=2`: Var((x_1+x_2+...X_T)/T)


"""
function CovNWFn(g0,m=0,DivideByT=0)

    T = size(g0,1)                    #g0 is Txq
    m = min(m,T-1)                    #number of lags

    g = g0 .- mean(g0,dims=1)         #normalizing to zero means

    S = g'g                           #(qxT)*(Txq)
    for s = 1:m
        Λ_s = g[s+1:T,:]'g[1:T-s,:]   #same as Sum[g_t*g_{t-s}',t=s+1,T]
        S   = S  +  (1 - s/(m+1))*(Λ_s + Λ_s')
    end

    (DivideByT > 0) && (S = S/T^DivideByT)

    return S

end

CovNWFn

In [7]:
S      = CovNWFn(X.*u,2)         #S acccording to Newey-West, 2 lags
V      = inv(Sxx)'S*inv(Sxx)     #Cov(b), Newey-West
std_NW = sqrt.(diag(V))

S       = CovNWFn(X.*u,0)         #S acccording to Newey-West, 0 lags
V       = inv(Sxx)'S*inv(Sxx)
std_NW0 = sqrt.(diag(V))

printblue("Coefficients and standard errors (from different methods):\n")
xx = [b std_iid std_W std_NW std_NW0]
printmat(xx,colNames=["b","std_iid","std_White","std_NW","std_NW 0 lags"],rowNames=xNames,width=16)

printred("Remark: NW with 0 lags should be the same as White's method")

Coefficients and standard errors (from different methods):

                  b         std_iid       std_White          std_NW   std_NW 0 lags
c             0.007           0.002           0.002           0.002           0.002
SMB           0.217           0.073           0.113           0.129           0.113
HML          -0.429           0.074           0.097           0.118           0.097

Remark: NW with 0 lags should be the same as White's method
